In [ ]:
import cv2
from scipy import misc
import numpy as np 
import matplotlib.pylab as plt
import argparse
import math
import os
from os import listdir
from PIL import Image
from random import seed
from random import randint
from __future__ import print_function
from IPython import display
import sys
import sys, json
import time

!git clone https://github.com/tensorflow/tpu                                            
display.clear_output()

import  eval_ckpt_main as eval_ckpt
import tensorflow.compat.v1 as tf

sys.path.append("/home/ubuntu/.jupyter/MyNotebooks/tpu/models/official/efficientnet")   #Change to your path
sys.path.append("/home/ubuntu/.jupyter/MyNotebooks/tpu/models/common")                  #Change to your path
model_name = "efficientnet-b4"                                                          #Change to your targetted model (e.g., eff_X)


In [ ]:
# Selecting Random sampled images from ImageNet-A

path3 = '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/'                      #Change to your path
dirs=os.listdir(path3)

# From random seed
seed(1)

selected_img = [0] * 7500

directories=[]
count=0
# Traversing directories 
for i in range(0, len(dirs)):
  dirs2=os.listdir(path3+dirs[i])
  for y in range(0, len(dirs2)):
    directories.append(path3+dirs[i]+'/'+dirs2[y])
    count+=1

for a in range(0, 3):
    for i in range(0, 150):
        value = randint(0, len(directories) - 1)
        while(selected_img[value]!=0):
            value = randint(0, len(directories) - 1)
        a_file = open("RandomDirectories"+str(a)+".txt", "a")
        selected_img[value]=1
        a_file.write(str(directories[value]) + "\n")
        print(directories[value]) 
        a_file.close()

In [ ]:
# Removing corrupted images -- filtering
def progress(percent, width=50):
    '''Progress printing function'''
    if percent >= 100:
        percent = 100

    show_str = ('[%%-%ds]' % width) % (int(width * percent / 100) * "#")  # Nested use of string splicing
    print('\r%s %d%% ' % (show_str, percent), end='')

def is_valid_jpg(jpg_file):
    with open(jpg_file, 'rb') as f:
        f.seek(-2, 2)
        buf = f.read()
        f.close()
        return buf ==  b'\xff\xd9'  # Determine whether the .jpg contains the end field

myFile = open("corrupted_images.txt", 'a')                                                      
fDir = '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/'                       #Change to your path
fSubDir = os.listdir(fDir)

for i in range(0, len(fSubDir)):
  train_dir = fDir + fSubDir[i] + '/'

  data_size = len([lists for lists in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, lists))])
  recv_size = 0
  incompleteFile = 0
  print('file tall : %d' % data_size)

  for file in os.listdir(train_dir):
      if os.path.splitext(file)[1].lower() == '.jpg':
          ret = is_valid_jpg(train_dir + file)
          if ret == False:
              incompleteFile = incompleteFile + 1
              print(train_dir + file + '\n')
              myFile.write(train_dir + file + '\n')
              os.remove(train_dir + file)

      recv_per = int(100 * recv_size / data_size)
      progress(recv_per, width=30)
      recv_size = recv_size + 1

  progress(100, width=30)
  print('\nincomplete file : %d' % incompleteFile)
myFile.close()

In [ ]:
# Reading directory and dir_labels from randomly generated files

path3 = '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/'                      #Change to your path
dirs=os.listdir(path3)

count=0
# Traversing directories 
for i in range(0, len(dirs)):
  dirs2=os.listdir(path3+dirs[i])
  for y in range(0, len(dirs2)):
    mainpath=path3+dirs[i]+'/'+dirs2[y]
  

# 1000-ImagenetA Labels 
d = {}
with open("/home/ubuntu/.jupyter/MyNotebooks/ImageNet-A_labels/imagenet1000_clsidx_to_labels.txt") as f:                #Change to your path
    for line in f:
       (key, val) = line.split(': ')
       d[int(key)] = val[1:-3]

# Imagenet-A Folder Labels 
folders = {}
with open("/home/ubuntu/.jupyter/MyNotebooks/ImageNet-A_labels/map_clsloc.txt") as f:                                   #Change to your path
    for line in f:
      a=line.split(' ')
      folders[a[0]] = a[2][:-1]


# Create 1st random directory
randdirs1=[]
with open("/home/ubuntu/.jupyter/MyNotebooks/RandomDirectories0.txt") as f:                                              #Change to your path
    for line in f:
       val = line.split('\n')
       randdirs1.append(val[0])  
 
# Create 2nd random directory
randdirs2=[]
with open("/home/ubuntu/.jupyter/MyNotebooks/RandomDirectories1.txt") as f:                                              #Change to your path
    for line in f:
       val = line.split('\n')
       randdirs2.append(val[0])
        
# Create 3rd random directory
randdirs3=[]
with open("/home/ubuntu/.jupyter/MyNotebooks/RandomDirectories2.txt") as f:                                              #Change to your path
    for line in f:
       val = line.split('\n')
       randdirs3.append(val[0])

['/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/n03775071/0.004684_kimono _ kimono_0.8802935.jpg', '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/n09229709/0.001222_candle _ candle_0.3989783.jpg', '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/n01687978/0.000860_syringe _ syringe_0.66711265.jpg', '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/n01580077/0.041174_unicycle _ unicycle_0.9613204.jpg', '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/n02410509/0.094490_snowmobile _ snowmobile_0.46591586.jpg', '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/n02236044/0.003268_stick insect _ stick insect_0.9985708.jpg', '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/n02280649/0.000325_beaker _ beaker_0.97071755.jpg', '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/n02133161/0.002153_red fox _ saxophone_0.4346895.jpg', '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/n01843383/0.015228_American robin _ American 

In [ ]:
# Functions that could be used to test the trained model without file i/o

# ----------------------.......----------------------
# def randomSample(imagedir):
#   if (len(imagedir) <= 6):
#       return imagedir   
#   else:
#       newimagedir = [] 
#       for i in range (0, 5):
#           value = randint(0, len(imagedir) - 1)
#           newimagedir.append(imagedir[value]) 
#       return newimagedir  


# ----------------------.......----------------------
# def calculateTopAccuracy(topAcc, totalNumPic):
#   topAccValues = [0.0, 0.0, 0.0, 0.0, 0.0]
#   for i in range(0, len(topAcc)): 
#     topAccValues[i] = topAcc[i]/totalNumPic 
#   return topAccValues


# ----------------------.......----------------------
# def printResult(model_name, topAcc, topAccValues, totalNumPic):
#     print("Model name: " + model_name)
#     print("Top Accuracy" + "\t" + "Correctly Predicted Ones" + "\t" + "TotalPic" + "\t" + "TotalAccVal") 
#     for i in range(0, len(topAcc)): 
#       print(i+1, "\t\t", topAcc[i], "\t\t\t\t", totalNumPic, "\t\t", topAccValues[i])

# Commonly used for all three following cells
# Returns true if predicted label matches with the class name
def isClassFound(folderLabel, possibleClasses):
    isFound = False
    if folderLabel in possibleClasses: 
          isFound = True
    return isFound

In [ ]:
# Running EfficientNet here

# seed random number generator
seed(1)

# Gets already trained checkpoints from Google API and our Goodle Drive
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckpts/{model_name}.tar.gz -O {model_name}.tar.gz  #Change to Google Drive link for AdvProp and NoisyStud
!tar xf {model_name}.tar.gz
ckpt_dir = model_name
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.json -O labels_map.json
labels_map_file = "/home/ubuntu/.jupyter/MyNotebooks/labels_map.json"                                                     #Change to your path
eval_driver     = eval_ckpt.get_eval_driver(model_name)

# Retrieve filtered ImagenetA dataset
imagepath = '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/'                                                    #Change to your path
imagedirs=os.listdir(imagepath)

# Test the images
totalNumofPic = 0   
topAcc        = [0, 0, 0, 0, 0]

for i in range(0, len(randdirs1)):
    file1 = open("results1"+model_name+".txt", "a")
    start_time = time.time()
    mainpath = randdirs1[i]
    print(randdirs1[i][56:65])
    totalNumofPic += 1
    image_files = [mainpath]
    print(mainpath)
    pred_idx, pred_prob  = eval_driver.eval_example_images(ckpt_dir, image_files, labels_map_file)
    folderNumber = randdirs1[i][56:65]
    folderLabel  = folders[folderNumber] 
    print(folderLabel)
    isFound  = False
    classnumArr = pred_idx[0]
    for k in range (0, 5): 
      classnum = classnumArr[k]
      possibleClasses = d[int(classnum)]
      if isFound == False:
        isFound      = isClassFound(folderLabel, possibleClasses)
        if isFound:
          topAcc[k] += 1    
      else: 
        topAcc[k]   += 1
    print(topAcc)
    print("--- %s seconds ---" % (time.time() - start_time))
    file1.write(mainpath+" "+ str(totalNumofPic) + " ")
    file1.write(str(topAcc))
    file1.write("\n")
    file1.close()

# Report results
file1 = open("results1"+model_name+".txt", "a") 
file1.write("Model name: " + model_name + "\n")
file1.write("Top Accuracy" + "\t" + "Correctly Predicted Ones" + "\t" + "TotalPic" + "\t" + "TotalAccVal" + " \n") 
for i in range(0, len(topAcc)): 
    file1.write(str(i+1)+ "\t\t"+ str(topAcc[i])+ "\t\t\t\t"+ str(totalNumofPic)+ "\n")
file1.close()

In [ ]:
# Running AdvProp here

# seed random number generator
seed(1)

# Gets already trained checkpoints from Google API and our Goodle Drive
!wget https://drive.google.com/drive/folders/18OZEOW1RFB_OI0_uI4F4qwBC3poGFeKU/{model_name}.tar.gz -O {model_name}.tar.gz
!tar xf {model_name}.tar.gz
ckpt_dir = model_name
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.json -O labels_map.json
labels_map_file = "/home/ubuntu/.jupyter/MyNotebooks/labels_map.json"                                                     #Change to your path
eval_driver     = eval_ckpt.get_eval_driver(model_name)

# Retrieve filtered ImagenetA dataset
imagepath = '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/'                                                    #Change to your path
imagedirs=os.listdir(imagepath)

# Test the images
totalNumofPic = 0   
topAcc        = [0, 0, 0, 0, 0]

for i in range(0, len(randdirs2)):
    file1 = open("advpropresults2"+model_name+".txt", "a")
    start_time = time.time()
    mainpath = randdirs2[i]
    print(randdirs2[i][56:65])
    totalNumofPic += 1
    image_files = [mainpath]
    print(mainpath)
    pred_idx, pred_prob  = eval_driver.eval_example_images(ckpt_dir, image_files, labels_map_file)
    folderNumber = randdirs2[i][56:65]
    folderLabel  = folders[folderNumber] 
    print(folderLabel)
    isFound  = False
    classnumArr = pred_idx[0]
    for k in range (0, 5): 
      classnum = classnumArr[k] 
      possibleClasses = d[int(classnum)]
      if isFound == False:
        isFound      = isClassFound(folderLabel, possibleClasses)
        if isFound:
          topAcc[k] += 1    
      else: 
        topAcc[k]   += 1
    print(topAcc)
    print("--- %s seconds ---" % (time.time() - start_time))
    file1.write(mainpath+" "+ str(totalNumofPic) + " ")
    file1.write(str(topAcc))
    file1.write("\n")
    file1.close()

# Report results
file1 = open("advpropresults2"+model_name+".txt", "a") 
file1.write("Model name: " + model_name + "\n")
file1.write("Top Accuracy" + "\t" + "Correctly Predicted Ones" + "\t" + "TotalPic" + "\t" + "TotalAccVal" + " \n") 
for i in range(0, len(topAcc)): 
    file1.write(str(i+1)+ "\t\t"+ str(topAcc[i])+ "\t\t\t\t"+ str(totalNumofPic)+ "\n")
file1.close()

In [ ]:
# Running NoisyStud here

# seed random number generator
seed(1)

# Gets already trained checkpoints from Google API and our Goodle Drive
!wget https://drive.google.com/drive/folders/1nE7pmsnWHbpLLc8HNiqmPzCUaJpO2lev/{model_name}.tar.gz -O {model_name}.tar.gz
!tar xf {model_name}.tar.gz
ckpt_dir = model_name
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.json -O labels_map.json
labels_map_file = "/home/ubuntu/.jupyter/MyNotebooks/labels_map.json"                                                     #Change to your path
eval_driver     = eval_ckpt.get_eval_driver(model_name)

# Retrieve filtered ImagenetA dataset
imagepath = '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/'                                                    #Change to your path
imagedirs=os.listdir(imagepath)

# Test the images
totalNumofPic = 0   
topAcc        = [0, 0, 0, 0, 0]

for i in range(0, len(randdirs3)):
    file1 = open("noisyresults3"+model_name+".txt", "a")
    start_time = time.time()
    mainpath = randdirs3[i]
    print(randdirs3[i][56:65])
    totalNumofPic += 1
    image_files = [mainpath] 
    print(mainpath)
    pred_idx, pred_prob  = eval_driver.eval_example_images(ckpt_dir, image_files, labels_map_file) 
    folderNumber = randdirs3[i][56:65]
    folderLabel  = folders[folderNumber] 
    print(folderLabel)
    isFound  = False
    classnumArr = pred_idx[0]
    for k in range (0, 5): 
      classnum = classnumArr[k] 
      possibleClasses = d[int(classnum)]
      if isFound == False:
        isFound      = isClassFound(folderLabel, possibleClasses)
        if isFound:
          topAcc[k] += 1    
      else: 
        topAcc[k]   += 1
    print(topAcc)
    print("--- %s seconds ---" % (time.time() - start_time))
    file1.write(mainpath+" "+ str(totalNumofPic) + " ")
    file1.write(str(topAcc))
    file1.write("\n")
    file1.close()

# Report results
file1 = open("noisyresults3"+model_name+".txt", "a") 
file1.write("Model name: " + model_name + "\n")
file1.write("Top Accuracy" + "\t" + "Correctly Predicted Ones" + "\t" + "TotalPic" + "\t" + "TotalAccVal" + " \n") 
for i in range(0, len(topAcc)): 
    file1.write(str(i+1)+ "\t\t"+ str(topAcc[i])+ "\t\t\t\t"+ str(totalNumofPic)+ "\n")
file1.close()

#Control testing on a single image if necessary#

In [ ]:
# WORKING SINGLE EXAMPLE EFFICIENTNET

name = '/home/ubuntu/.jupyter/MyNotebooks/imagenet-a/imagenet-a/n01498041/0.000348_chameleon _ box turtle_0.55540705.jpg'
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckpts/{model_name}.tar.gz -O {model_name}.tar.gz  #Change to Google Drive link for AdvProp and NoisyStud
!tar xf {model_name}.tar.gz
ckpt_dir = model_name
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.json -O labels_map.json
labels_map_file = "/content/labels_map.json"
image_file = name
image_files = [image_file]
eval_driver = eval_ckpt.get_eval_driver(model_name)
pred_idx, pred_prob  = eval_driver.eval_example_images(ckpt_dir, image_files, labels_map_file)
print(pred_idx)
